In [ ]:
'''importing module'''
import numpy as np 
import pandas as pd 
import matplotlib                  
import matplotlib.pyplot as plt
import seaborn as sns              
plt.style.use('fivethirtyeight')
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Import and preparing data

summary_data = pd.read_csv('/kaggle/input/air-pollution-in-seoul/AirPollutionSeoul/Measurement_summary.csv')
summary_data.drop(['Address', 'Latitude', 'Longitude'], axis=1, inplace=True)
summary_data['Measurement date'] = pd.to_datetime(summary_data['Measurement date'])
negative_data = summary_data[(summary_data.SO2 < 0) | (summary_data.NO2 < 0) | (summary_data.O3 < 0) | (summary_data.CO < 0) | (summary_data.PM10 < 0) | (summary_data['PM2.5'] < 0)]
summary_data.drop(negative_data.index, axis=0, inplace=True)
summary_data = summary_data[summary_data['Station code'] == 101]
summary_data.drop('Station code', axis=1, inplace=True)
summary_data.reset_index(drop=True, inplace=True)
summary_data.head()

# PM2.5 Linear Forest

In [ ]:
csv_data = summary_data.to_csv(index=False)

In [ ]:
with open('111.csv', 'w') as f:
    f.write(csv_data)

In [ ]:
summary_data = summary_data[['Measurement date', 'PM2.5']]
summary_data

In [ ]:
x = summary_data[summary_data['PM2.5'] > 100]

In [ ]:
summary_data['day'] = summary_data['Measurement date'].dt.day_name()
summary_data['month'] = summary_data['Measurement date'].dt.month_name()
summary_data['hour'] = summary_data['Measurement date'].dt.hour
summary_data

In [ ]:
x['day'] = x['Measurement date'].dt.day_name()
x['month'] = x['Measurement date'].dt.month_name()
x['hour'] = x['Measurement date'].dt.hour
x

In [ ]:
x.groupby('day').count()

In [ ]:
data_one_hot = pd.get_dummies(summary_data, columns=['day', 'month', 'hour'])
data_one_hot

In [ ]:
split_point = round(len(data_one_hot)*0.8)
data_one_hot_train = data_one_hot[:split_point].copy()
data_one_hot_test = data_one_hot[split_point:].copy()

In [ ]:
data_one_hot_test

In [ ]:
X_train = data_one_hot_train.drop(['Measurement date', 'PM2.5'], axis=1)
y_train = data_one_hot_train['PM2.5']
X_test = data_one_hot_test.drop(['Measurement date', 'PM2.5'], axis=1)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
from sklearn.ensemble import RandomForestRegressor 

In [ ]:
model = RandomForestRegressor(200)
model.fit(X_train, y_train)

In [ ]:
y_predict = model.predict(X_test)
y_predict

In [ ]:
data_one_hot_test['predict'] = y_predict
data_one_hot_test

In [ ]:
plt.figure(figsize=(25, 6))
sns.lineplot(x='Measurement date', y='PM2.5', data=data_one_hot_test, linewidth=1)
sns.lineplot(x='Measurement date', y='predict', data=data_one_hot_test, linewidth=1)
plt.ylim((0, 100))
plt.legend(['PM2.5', 'predict'])

In [ ]:
np.mean(data_one_hot_test['PM2.5'])

In [ ]:
np.mean(data_one_hot_test['predict'])

In [ ]:
np.mean(abs(data_one_hot_test['PM2.5'] - data_one_hot_test['predict']))

In [ ]:
data_one_hot['predict'].mean()